In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import os
import datetime

# Data - Amazon Movie Reviews - 9.33 GB >> a few sub documents can be generated
# /Users/melih/Documents/Masters/KTH_DASE-ICT Innovation_Data Science/7.Data Mining/Project/test_graph/KTH-ID2211-Data-Mining-master/data/movies.txt

In [2]:
class Shingling:
    def Compute_Shingle_Set_string(self, input_string, k_shingles = 10, run_mode = 'normal'):
        total_characters = len(input_string)
        character_processing = 0
        shingle_set = list()
        shingle_set_hash = list()
        
        # SETs are unordered,that's why we do our own check for the uniqueness
        #https://docs.python.org/2/library/sets.html
        while (character_processing + k_shingles) < total_characters + 1:            
            shingle = input_string[character_processing: character_processing + k_shingles]
            #print(shingle)
            if shingle not in shingle_set:
                shingle_set.append(shingle)
                shingle_set_hash.append(hash(shingle))
            character_processing += 1
            
        if run_mode == 'debug':
            print(shingle_set)
        
        return shingle_set_hash
    

In [3]:
class CompareSets:
    # not necessarily need this
    def Create_Global_Set(self, set_list):
        set_global = []
        for set_x in set_list:
            set_global += set_x
        
        set_global_unique = np.unique(set_global)
        
        return set_global_unique
    
    def Compare_2_Sets(self, set1, set2):
        common_shingles = 0
     
        set_union = set1 + set2
        set_union_len = len(set_union)
        set_unique = np.unique(set_union)
        set_unique_size = set_unique.size
        
        # (set_union_len - set_unique_size) gives the intersection = duplicates
        Jaccard_similarity = (set_union_len - set_unique_size)/set_unique_size
        
        return Jaccard_similarity
    
    def Compare_2_Sets_with_GlobalSet(self, set1, set2):
        common_shingles = 0
      
        set_union = set1 + set2
        set_union_len = len(set_union)
        set_unique = np.unique(set_union)
        set_unique_size = set_unique.size
        
        # (set_union_len - set_unique_size) gives the intersection = duplicates
        Jaccard_similarity = (set_union_len - set_unique_size)/set_unique_size
        
        return Jaccard_similarity, set_unique
    
    def Compare_Sets_Global(self, set1, set2, set_global_unique):      
        set_union = set1 + set2
        set_union_len = len(set_union)
        set_unique = np.unique(set_union)
        set_unique_size = set_unique.size
     
        # create 0/1 set
        set_1_global = [1 if s in set1 else 0 for s in set_global_unique]
        set_2_global = [1 if s in set2 else 0 for s in set_global_unique]
        
        # (set_union_len - set_unique_size) gives the intersection = duplicates
        Jaccard_similarity = (set_union_len - set_unique_size)/set_unique_size
        
        return Jaccard_similarity, set_1_global, set_2_global
    
    def Generate_Sets_Global_01(self, set_list, set_global_unique):      
        set_01 = []
        # for each set:
        # for each shingle in the given set (document), check the global set.
        # Then create a new set in the same length of the global set by setting 0 if the global shingle is not in
        # the given set, or by setting 1 if the global shingle is in the given set
        # finally we will have #docs (12 in our case) of sets with 0/1 values each having the same length
        # with the global shingle set
        for i, set1 in enumerate(set_list):
            print(">>>> Generating 0/1 for set#{}... time: {}".format(i, datetime.datetime.now()))
            set_1_01 = [1 if s in set1 else 0 for s in set_global_unique]
            set_01.append(set_1_01)
        
        return set_01

In [4]:
class MinHashing:        
    def Generate_Hash_Functions(self, len_set_global_unique, number_of_hashfunctions = 100):
        # format: (ax + b) % c
        # so we will need to pass (a, b, c) values
        # c = len_set_global_unique = total number of unique shingles
        # assuming that c will be equal to the len(set_global_unique), we need to pass (a, b)
        hash_func_params = []
        
        total_funcs_generated = 0
        while total_funcs_generated < number_of_hashfunctions:
            a = random.randint(1, len_set_global_unique - 1)
            b = random.randint(1, len_set_global_unique - 1)
            if (a, b) not in hash_func_params:
                hash_func_params += [(a, b)]
                total_funcs_generated += 1
        
        return hash_func_params   
        
    # creates a signature for a set/document = set_1_global using hash functions with the provided params
    def CreateMinHash_Signature(self, set_1_global, hash_func_params):
        # inifinity for us is equal to any number greater than or equal to "len_set_global_unique"
        # Because we will do a min(x ,y) comparison and the function value will be moded to the "len_set_global_unique"
        # "len_set_global_unique" can be chosen as the max value/infinity for our application
        len_set_global_unique = len(set_1_global)
        # we create an array with the length of the global set, and set the values of each item to the infinity
        # (infinity == len_set_global_unique) in our case
        set_1_signature = np.full(len(hash_func_params), len_set_global_unique)
        
        # here, we are handling 1 document only
        # for each row (shingle value) in the given set, we calculate the hash_function result
        # if the value is not 0 (in other words if the value == 1)
        # then we compare it with the existing value in the signature. 
        # If the new value is smaller than the previous value, we update the signature
        for row in range(len_set_global_unique):
            if set_1_global[row] != 0:
                for h, params in enumerate(hash_func_params):
                    hash_value = (row*params[0] + params[1]) % len_set_global_unique
                    set_1_signature[h] = min(set_1_signature[h], hash_value)
        
        return set_1_signature

In [5]:
class CompareSignatures:
    def Find_Similarity(self, set_1_signature, set_2_signature):
        count_similar = 0
        len_set_1_signature = len(set_1_signature)

        for i in range(len_set_1_signature):
            if set_1_signature[i] == set_2_signature[i]:
                count_similar += 1

        sim_s1_s2 = count_similar/len_set_1_signature

        return sim_s1_s2

In [6]:
def Read_File_2String(file_path):
    file_read = open(file_path, "r")
    file_context = file_read.read()
    file_read.close()
    return file_context

In [7]:
# we mention the directory where we store the docs to be compared
data_directory = 'data_1/'
document_list = os.listdir(data_directory)
document_list_shingle_set = []

shingle_obj = Shingling()
cs = CompareSets()
mh = MinHashing()
comp_sig = CompareSignatures()
n_hash_functions = 100

for doc in document_list:
    file_path = data_directory + doc    
    str_doc = Read_File_2String(file_path)
    print(">>>> File {}, creating shingles... time: {}".format(file_path, datetime.datetime.now()))
    shingle_doc = shingle_obj.Compute_Shingle_Set_string(str_doc, k_shingles = 10)
    document_list_shingle_set.append(shingle_doc)

print(">>>> Generating the Global Set (unique shingles)... time: {}".format(datetime.datetime.now()))
unique_shingles_global = cs.Create_Global_Set(document_list_shingle_set)
c = len(unique_shingles_global)  # len_set_global_unique

print('>>>> n_hash_functions: {}, c: {}'.format(n_hash_functions, c))

print(">>>> Generating 0/1 Sets for all docs... time: {}".format(datetime.datetime.now()))
document_list_set_01 = cs.Generate_Sets_Global_01(document_list_shingle_set, unique_shingles_global)

print(">>>> Generating Hash Functions to use... time: {}".format(datetime.datetime.now()))
hash_func_params = mh.Generate_Hash_Functions(c, n_hash_functions)

print(">>>> Creating the signatures per doc... time: {}".format(datetime.datetime.now()))
signature_list = []
for i, set_x in enumerate(document_list_set_01):
    signature_list.append(mh.CreateMinHash_Signature(set_x, hash_func_params))
        

>>>> File data_1/Movies_5.11.2020_18.54.20_deleted_MORE_words.txt, creating shingles... time: 2020-11-09 17:46:04.772954
>>>> File data_1/Movies_5.11.2020_18.54.19.txt, creating shingles... time: 2020-11-09 17:46:04.788010
>>>> File data_1/Movies_5.11.2020_18.54.18.txt, creating shingles... time: 2020-11-09 17:46:04.963153
>>>> File data_1/Movies_5.11.2020_18.54.23.txt, creating shingles... time: 2020-11-09 17:46:05.048573
>>>> File data_1/Movies_5.11.2020_18.54.22.txt, creating shingles... time: 2020-11-09 17:46:05.267275
>>>> File data_1/Movies_5.11.2020_18.54.20.txt, creating shingles... time: 2020-11-09 17:46:05.297802
>>>> File data_1/Movies_5.11.2020_18.54.21.txt, creating shingles... time: 2020-11-09 17:46:05.312274
>>>> File data_1/Movies_5.11.2020_18.54.20_deleted_some_words.txt, creating shingles... time: 2020-11-09 17:46:05.441541
>>>> File data_1/Movies_5.11.2020_18.54.16.txt, creating shingles... time: 2020-11-09 17:46:05.462353
>>>> File data_1/Movies_5.11.2020_18.54.17.t

In [8]:
# >>>>>> JACCARD SIMILARITY MATRIX vs 
# >>>>>> SIGNATURE SIMILARITY MATRIX
#cs = CompareSets()
n_document = len(document_list)
Matrix_Jaccard_similarity = np.zeros((n_document, n_document))
Matrix_SIG_similarity = np.zeros((n_document, n_document))

for d1 in range(n_document):
    for d2 in range(n_document):
        # since the similarity will be symmetric, we don't need to calculate it twice
        if d1 <= d2:
            js = cs.Compare_2_Sets(document_list_shingle_set[d1], document_list_shingle_set[d2])
            Matrix_Jaccard_similarity[d1][d2] = round(js, 3)
            Matrix_Jaccard_similarity[d2][d1] = Matrix_Jaccard_similarity[d1][d2]
            Matrix_SIG_similarity[d1][d2] = comp_sig.Find_Similarity(signature_list[d1], signature_list[d2])
            Matrix_SIG_similarity[d2][d1] = Matrix_SIG_similarity[d1][d2]
            
            if Matrix_Jaccard_similarity[d1][d2] >= 0.8 and d1 != d2:
                print("\nJACCARD: (d{}, d{}, {}): {} similar to {} by the ratio: {}".format(d1, d2, Matrix_Jaccard_similarity[d1][d2], document_list[d1], document_list[d2], Matrix_Jaccard_similarity[d1][d2]))
            if Matrix_SIG_similarity[d1][d2] >= 0.8 and d1 != d2:
                print("SIG: (d{}, d{}, {}): {} similar to {} by the ratio: {}".format(d1, d2, Matrix_SIG_similarity[d1][d2], document_list[d1], document_list[d2], Matrix_SIG_similarity[d1][d2]))
        
print("\nJACCARD SIMILARITY MATRIX:\n{}".format(Matrix_Jaccard_similarity))
print("\nSIGNATURE SIMILARITY MATRIX:\n{}".format(Matrix_SIG_similarity))



JACCARD: (d0, d5, 0.896): Movies_5.11.2020_18.54.20_deleted_MORE_words.txt similar to Movies_5.11.2020_18.54.20.txt by the ratio: 0.896
SIG: (d0, d5, 0.89): Movies_5.11.2020_18.54.20_deleted_MORE_words.txt similar to Movies_5.11.2020_18.54.20.txt by the ratio: 0.89

JACCARD: (d0, d7, 0.934): Movies_5.11.2020_18.54.20_deleted_MORE_words.txt similar to Movies_5.11.2020_18.54.20_deleted_some_words.txt by the ratio: 0.934
SIG: (d0, d7, 0.94): Movies_5.11.2020_18.54.20_deleted_MORE_words.txt similar to Movies_5.11.2020_18.54.20_deleted_some_words.txt by the ratio: 0.94

JACCARD: (d5, d7, 0.956): Movies_5.11.2020_18.54.20.txt similar to Movies_5.11.2020_18.54.20_deleted_some_words.txt by the ratio: 0.956
SIG: (d5, d7, 0.95): Movies_5.11.2020_18.54.20.txt similar to Movies_5.11.2020_18.54.20_deleted_some_words.txt by the ratio: 0.95

JACCARD SIMILARITY MATRIX:
[[1.    0.03  0.043 0.021 0.045 0.896 0.035 0.934 0.033 0.018 0.017 0.013]
 [0.03  1.    0.032 0.02  0.024 0.03  0.035 0.03  0.019 0.

In [9]:
# LSH - Locality Sensitive Hashing
band_buckets = []
n_band = 20
r = 5
#hash_temp = hash(str(list(np.zeros(100, int))))

for b in range(n_band):
    band_bucket_id = []
    band_bucket_docid = []
    for i, sig in enumerate(signature_list):
        hash_temp = hash(str(signature_list[i][b:b+r]))
        if hash_temp not in band_bucket_id:
            band_bucket_id.append(hash_temp)
            band_bucket_docid.append([i])
        else:
            bucket_id = band_bucket_id.index(hash_temp)
            band_bucket_docid[bucket_id].append(i)
    
    band_buckets.append([b, band_bucket_docid])
                        

In [10]:
band_buckets

[[0, [[0], [1], [2], [3], [4], [5, 7], [6], [8], [9], [10], [11]]],
 [1, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [2, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [3, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [4, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [5, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [6, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [7, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [8, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [9, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [10, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [11, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [12, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [13, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [14, [[0, 5, 7], [1], [2], [3], [4], [6], [8], [9], [10], [11]]],
 [1

In [11]:
for i, doc in enumerate(document_list):
    file_path = data_directory + doc
    str_doc = Read_File_2String(file_path)
    print("d{}: {} has {} characters".format(i, doc, len(str_doc)))

d0: Movies_5.11.2020_18.54.20_deleted_MORE_words.txt has 1808 characters
d1: Movies_5.11.2020_18.54.19.txt has 6053 characters
d2: Movies_5.11.2020_18.54.18.txt has 4300 characters
d3: Movies_5.11.2020_18.54.23.txt has 6559 characters
d4: Movies_5.11.2020_18.54.22.txt has 2591 characters
d5: Movies_5.11.2020_18.54.20.txt has 2008 characters
d6: Movies_5.11.2020_18.54.21.txt has 5298 characters
d7: Movies_5.11.2020_18.54.20_deleted_some_words.txt has 1966 characters
d8: Movies_5.11.2020_18.54.16.txt has 2796 characters
d9: Movies_5.11.2020_18.54.17.txt has 8348 characters
d10: Movies_5.11.2020_18.54.15.txt has 6717 characters
d11: Movies_5.11.2020_18.54.14.txt has 10270 characters


In [12]:
print(hash_func_params)

[(34161, 24848), (24050, 2570), (37414, 11497), (11174, 13063), (40991, 2879), (16805, 14204), (34324, 23839), (33270, 30855), (41558, 28641), (24005, 18379), (43550, 24780), (9954, 5634), (9820, 20219), (22965, 42033), (9507, 29827), (392, 21812), (12960, 28913), (11773, 6631), (37323, 35648), (33881, 31943), (41244, 27213), (185, 14624), (36122, 43387), (6883, 22232), (1714, 6332), (16890, 18790), (32709, 39189), (26177, 7397), (31619, 25409), (32477, 38090), (2853, 5560), (43575, 3955), (28262, 23840), (37653, 2088), (35143, 27116), (23996, 14345), (2861, 9462), (39461, 36042), (13216, 37193), (24257, 20586), (40854, 27638), (14274, 39960), (1189, 7171), (15305, 14494), (2222, 23414), (22284, 6834), (11022, 43655), (22910, 1004), (35568, 7520), (27885, 4626), (4459, 36789), (5403, 41326), (7922, 1738), (33890, 15659), (13289, 16208), (41850, 30511), (25298, 16944), (27088, 35576), (8224, 24936), (31747, 34641), (15313, 43576), (29059, 11996), (18625, 20990), (36667, 40322), (15024, 

In [13]:
signature_list[1]

array([ 3,  2, 13,  3,  1, 28, 11,  3,  1,  1, 10,  4,  7,  1,  4,  4, 17,
        2,  4,  2, 13,  1, 11, 13, 18,  8, 18,  9,  3, 12,  6, 19,  4, 18,
       33,  1,  4,  3,  9,  8,  8,  2,  7,  7, 14,  2,  1,  4,  0,  0, 31,
        1,  0, 29, 16,  7,  4,  8,  8, 13,  6, 15,  9,  9,  1,  4, 15,  0,
        2,  1, 15,  2, 12,  5, 12,  4, 12,  5,  5,  1,  5,  2,  0, 11,  7,
        8, 29,  2,  2,  6,  2,  0,  5,  9, 10,  8, 12,  1,  0, 10])

In [14]:
signature_list

[array([  7,  14,  43,  29,   0,  69,  23,  33,   5,  13,  14,  66,  43,
         22,  94,  44,  17,  11,  12,   8,  21,   9,   7,  45,  34,  50,
         37,   9,  49,   1,  38,  38, 100,  21,  54,  49,  41,  23,   9,
         26,  22,  24,   1,  26,  94,  30,  51,   4,   0,  25,  32,  69,
        124,   5,  27,  59,  36,   8,   8,  59,  47,  14,  31,   5,   1,
         34,  79,  98,  40,  72,  79,  10,   2,  22,  12, 169,   0,   2,
         28,  14,  37,  23,   0,   3,  19,  29,  38,  51,  20,   6,  47,
        100,  33,  39,  11,  32,  10, 101,  68,  72]),
 array([ 3,  2, 13,  3,  1, 28, 11,  3,  1,  1, 10,  4,  7,  1,  4,  4, 17,
         2,  4,  2, 13,  1, 11, 13, 18,  8, 18,  9,  3, 12,  6, 19,  4, 18,
        33,  1,  4,  3,  9,  8,  8,  2,  7,  7, 14,  2,  1,  4,  0,  0, 31,
         1,  0, 29, 16,  7,  4,  8,  8, 13,  6, 15,  9,  9,  1,  4, 15,  0,
         2,  1, 15,  2, 12,  5, 12,  4, 12,  5,  5,  1,  5,  2,  0, 11,  7,
         8, 29,  2,  2,  6,  2,  0,  5,  9, 10,  8, 12